In [ ]:
import numpy as np
import rioxarray
from xrspatial.zonal import stats
from rasterio.windows import from_bounds
import rasterio
import xarray as xr
from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCluster(n_workers=8)
client = Client(cluster)
client.dashboard_link

In [ ]:
LandscapeLayer = 'O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Projects/StreamCat/LandscapeRasters/QAComplete/CanalsDitches.tif'
inZoneData = 'O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Resource/Physical/HYDROLOGY/NHDPlusV21/NHDPlusPN/NHDPlus17/NHDPlusCatchment/cat'

In [ ]:
izd_array = rioxarray.open_rasterio(inZoneData, chunks=True).sel(band=1).drop_vars('band')
izd_array

In [ ]:
transform = izd_array.rio.transform()
bounds = izd_array.rio.bounds()
window = from_bounds(*bounds, transform)
window

In [ ]:
# with rasterio.open(LandscapeLayer) as src:
#     ll_array = src.read(1, window=window)
# ll_array
# ll_xarray =  xr.DataArray(
#         data=ll_array,
#         dims=['y', 'x']
#     ).chunk(izd_array.chunksizes)
# ll_xarray

In [ ]:
ll_array = rioxarray.open_rasterio(LandscapeLayer).sel(band=1).drop_vars('band')
ll_array

In [ ]:
ll_array_clipped = ll_array.rio.isel_window(window)
ll_array_clipped

In [ ]:
# Convert clipped array to dask array by chunking like izd_array
ll_array_clipped = ll_array_clipped.chunk(izd_array.chunksizes)

In [ ]:
outTable = stats(izd_array, ll_array_clipped, nodata_values=izd_array.rio.nodata)
outTable

In [ ]:
%%time
final_df = outTable.compute()

In [ ]:
final_df.head()

In [ ]:
final_df.to_csv('medium_res_stats_dask_test.csv')